
    
    
## <H2 align="center"> <font color='white'> **MAF1731B**: Microstructure and Trading Systems </H2>

### <H3 align="center"> <font color='white'> **Laboratory 4**: Market Microstructure </H3>

 <H5 align="center"> <em> Professor: Villalobos Ramos Omar Antonio <em>

 <p align="center"> <img style=" margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="350" height="150" /> 

 <strong> <H5 align="left"> Student:
Esteban Márquez Delgado <strong>

 *Registry*:
if700637: https://github.com/EstebanMqz/MyST_LAB_4

 <center> <font color= #555555> <font size = 4> November 2022 </a> <font color= #555555> <font size = 4> </a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Laboratory 4 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure  </font>

<hr style="border:0.02in solid gray"> </hr>

### <font color= #555555> <font size = 4> Abstract </font>

This document was prepared as delivery for Laboratory 4 of Microstructure and Trading Systems - MAF1731B class during the Autumn 2022 course at ITESO for the Bachelor of Financial Engineering. In order to analyze the market microstructure in criptocurrencies markets containing the following 3 sections:

1. **Consume CCXT data**: Download Criptocurrencies Order Books from exchanges (kraken, ftx, currencycom, coinmate, etc).

2. **Microstructure Visualization**: Build Time Series using the Order Books obtained in the previous point from the following elements of the Market Microstructure for each asset and exchange (Levels, Bid Volume, Ask Volume, Total Volume, Mid Price, VWAP.)

3. **Microstructure Modeling**: Estimation of Roll Effective Spread using the lagged series at t-5.



<font color= lightblue> <font size = 4> 0. Libraries and Dependencies: </font>

<font color= gray>

**Note**: To run this notebook it is necessary to have the following libraries contained in the [requirements.txt](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/requirements.txt) script of this project.

To install the libraries in the script run the following cell:

In [1]:
!pip install -r requirements.txt

ERROR: Invalid requirement: '"""' (from line 1 of requirements.txt)


If you prefer, manual installation can be done with the individual installation of the following libraries:

<font color= lightblue> <font size = 1> 
+ pandas >= 1.3.4
+ numpy >= 1.19.1
+ jupyter >= 1.0.0
+ chart_studio >= 1.1
+ plotly >= 4.14
+ datetime >= 4.7
+ pandas_datareader >= 0.10.0
+ tk >= 0.10
+ plotly >= 5.60
+ yahoofinancials >= 1.60
+ scipy >= 1.7.3
+ regex >= 2022.3.15
+ matplotlib >= 3.5.1
+ fire >= 0.4.0
+ ccxt >= 2.1.33
+ time >= 3.11 
+ logging >= 3.11

### <font color= lightblue> <font size = 4> 0.1 Introduction: </font>
In this laboratory we will use information from the Cryptocurrency Order Book of various international exchanges, with the primary objective of visualizing and modeling the microstructure of the market through the behavior of the Order Book and its characteristics such as; Matching Engine, Levels, Ask-Bid Volumes, etc.
<font color= gray>




##### *General help:*
In order, to retrieve specific help from the scripts tree in the project execute the help function on the script or in specific modules from the following:
+ [data.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/data.py)
+ [functions.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/functions.py)
+ [visualizations.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/visualizations.py)

In [2]:
## Libraries
import pandas as pd # Analysis and data management
import pandas_datareader as pdr
import pandas_datareader.data as web #Data extraction from the web
import numpy as np
from datetime import datetime, timedelta, date
from tkinter.ttk import Style
from tkinter import Y
import plotly.graph_objects as go #Plotly
import plotly.express as px
import warnings
from yahoofinancials import YahooFinancials # Yfinance 
import yfinance as yf
import ccxt #Criptocurrencies
import logging as log #Exchanges 
from scipy.optimize import minimize # Optimization 
from collections import Counter #Counter
import time
import re
import io 
import glob
import os
#.Py archives in repo
import functions as fn 
import visualizations as vs
import data as dt
from os import path
import fire
%matplotlib inline

<font color= orange> <font size = 5> 1. Consume CCXT data: </font>

In [3]:
help(fn.find_exchanges)

Help on function find_exchanges in module functions:

find_exchanges(features=None, is_authenticated=False)
    Function that returns avaliable cryptocurrencies exchanges in Python for CCTX module.
    
        Parameters:
        ----------
        features: None 
        is_authenticated: None 
    
        Returns:
        -------
        exchange_names: Array of authenticated exchanges avaliable in CCTX.



In [4]:
fn.find_exchanges()[:10] #len 120

['aax',
 'alpaca',
 'ascendex',
 'bequant',
 'bibox',
 'bigone',
 'binance',
 'binancecoinm',
 'binanceus',
 'binanceusdm']

In [5]:
exchanges= ['binance','ftx', 'bytetrade'] #Selected exchanges from list
criptos = ['BTC/USDT','ETH/USDT','XRP/USDT'] #Criptos OBs (modify tickers here)

<font color= lightblue> <font size = 4> 1.1 Order Book Levels (Bids/Asks) & OHLCV: </font>

In [6]:
help(vs.cctx_download)

Help on function cctx_download in module visualizations:

cctx_download(lvls, cripto, exchange)
    Function that returns prices and quantities of given levels (as integer) Bids & Asks in an Orderbook
    for the specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and
    from the following exchanges: binance, ftx or ascendex (as string).
    
        Parameters:
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable ticker (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        Returns:
        -------
        levels_ob_bid: Prices and Quantities of Bids (pos [0]) and Asks (pos [1]) as dataframes.



n levels downloaded by OB for given criptocurrency and exchange.

+ <font color= lightgray> <font size = 2> *Bids*: </font>

In [7]:
params= 25, criptos[1], exchanges[0]
params


(25, 'ETH/USDT', 'binance')

In [8]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,1483.01,1.2214
1,1482.91,7.9984
2,1482.89,2.6686
3,1482.88,5.5000
4,1482.87,6.3442


+ <font color= lightgray> <font size = 2> *Asks*: </font>

In [9]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,1483.02,14.7559
1,1483.06,0.0071
2,1483.07,0.0500
3,1483.09,9.0440
4,1483.10,11.8280


In [10]:
help(vs.OBLvls_hist)

Help on function OBLvls_hist in module visualizations:

OBLvls_hist(lvls, cripto, exchange)
    Function that plots an horizontal histogram in plotly for CriptoCurrencies OB/
    
        Parameters
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable ticker (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        x: Quantity (col) of the given cripto (str) for given lvls (int) in Order Book.
        y: Prices (col) of the given cripto (str) for given lvls (int) in Order Book.
    
        Returns
        -------
        Histogram of Order Book prices and quantities for n given lvls.



In [11]:
vs.OBLvls_hist(params[0],params[1], params[2])

<font color= lightblue> <font size = 4> 1.1.2 OHLCV: </font>

In [12]:
cols_ohlcv = ['exchange','timestamp', 'open', 'high', 'low', 'close', 'volume']
binance = ccxt.binance()
ohlcv=binance.fetch_ohlcv('BTC/USDT', limit=1)
ohlcv

ohlcv_df=pd.DataFrame(ohlcv, columns=cols_ohlcv[1:])
ohlcv_df['timestamp']=binance.iso8601(binance.milliseconds())
ohlcv_df
        

,timestamp,open,high,low,close,volume
0,2022-11-08T07:03:02.096Z,19773.65,19774.24,19772.48,19773.81,4.84216


<font color= lightblue> <font size = 4> 1.3 OB metrics: </font>

In [13]:
help(vs.Micro)

Help on function Micro in module visualizations:

Micro(lvls, cripto, exchange, n)
    Function that plots a line+marker graph with plotly.
    
        Parameters
        ----------
        x: index from Dataframe of selected metric to graph with plotly.
        y: Values of the selected of selected metric to graph with plotly.
        title: Title of the plot.
        x_label: Variable name in the label x.
        y_label: Variable name in the label y.
    
        Returns
        -------
        Returns a didactic graph with plotly of the selected metric.



In [31]:
MS = vs.Micro(params[0],params[1], exchanges[0], 3)

In [32]:
MS[0]

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
Order_Books,,,,,,,,
0,binance,2022-11-08T07:10:03.141,25,152.2928,75.1643,227.4571,1482.255,1488.766657
1,binance,2022-11-08T07:10:05.612,25,145.7606,91.3054,237.0660,1481.625,1487.869863
2,binance,2022-11-08T07:10:08.177,25,169.2403,88.3514,257.5917,1482.155,1487.903912


In [16]:
MS[1]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-08T07:03:03.563,1483.02,1483.51,1483.01,1483.50,16.7332
1,binance,2022-11-08T07:03:06.141,1483.02,1483.93,1483.01,1483.93,68.9953
2,binance,2022-11-08T07:03:08.789,1483.02,1483.93,1483.01,1483.50,102.2707


+ <font color= lightgray> <font size = 2> *OB Visualization*: </font>

<font color= orange> <font size = 5> 2. Microstructure Visualization </font>

<font color= lightblue> <font size = 4> 2.1 Dataframes </font>

<font color= orange> <font size = 5> 3. Microstructure Modelling: </font>

### <font color= orange> <font size = 5> 4. Bibliography: </font>

<font color= lightgray>

+ Villalobos, O, 2021. **Python Project Template**. Available at: https://github.com/OmarVillalobos/python-project. 
(Accessed: November 5, 2022). 

+ Financial Engineering, **Marketmicrostructure.pptx**. ITESO -  Microstructure and Trading Systems (ITE1731 - DMAF). 
Available at: https://docs.google.com/presentation/d/1uYyMERKKGeZ0MPqbJ3bcir9lfpzbeAH9ZMmxbYysokQ/edit?usp=sharing (Accessed: November 5, 2022). 

+ Muñoz E., J **Roll Model** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/Roll%20.pdf (Accessed: November 5, 2022).

+ Muñoz E., J **Asset Pricing Theory** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/APT.pdf (Accessed: November 5, 2022). 